This notebook exercises a bug in parameter by:
- Loading a data test
- Create a custom eval funciton (not sure if this is important or not)
- Running a a search with 100 permunations
- Waiting until the job completes by watching CPU usage
- Confirming the kernel is ok
- Running get_status and the kernal dies.

Viewing the logs its looks like all the tasks complete ok.

In [1]:
import graphlab as gl
gl.canvas.set_target('ipynb')
import tools 
import initialise

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.8.4.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.
2016-03-18 14:31:28,171 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.4 started. Logging: /tmp/graphlab_server_1458271886.log


This non-commercial license of GraphLab Create is assigned to kevin.mcisaac@gmail.com and will expire on November 06, 2016. For commercial licensing options, visit https://dato.com/buy/.


## Automatic optimisation

In [2]:
train_data, test_data = initialise.load_data(reload_data=False)
folds = gl.cross_validation.KFold(train_data, 5)

Loading saved processed data
306  features in total


In [3]:
def auc_eval(model, train, test): 
    return {'train_auc':    model.evaluate(train, metric='auc')['auc'],
            'validate_auc': model.evaluate(test,  metric='auc')['auc']}

#

In [4]:
params = {'target':'TARGET', 'random_seed':8923,
          'class_weights':'auto','metric':'auc',
          'early_stopping_rounds':10, 'max_iterations':300,
          'step_size': [0.01, 0.3],
          'max_depth': [3, 4, 5, 6, 7],
          'column_subsample': [0.8, 0.6]}

job = gl.random_search.create(folds,gl.boosted_trees_classifier.create,
                             params, evaluator=auc_eval, max_models=10, perform_trial_run=False)

2016-03-18 14:31:31,380 [INFO] graphlab.deploy.job, 22: Validating job.
2016-03-18 14:31:31,387 [INFO] graphlab.deploy.job, 36: Creating a LocalAsync environment called 'async'.
2016-03-18 14:31:31,397 [INFO] graphlab.deploy.map_job, 186: Validation complete. Job: 'Model-Parameter-Search-Mar-18-2016-14-31-3100000' ready for execution
2016-03-18 14:31:31,541 [INFO] graphlab.deploy.map_job, 192: Job: 'Model-Parameter-Search-Mar-18-2016-14-31-3100000' scheduled.
2016-03-18 14:31:31,720 [INFO] graphlab.deploy.job, 22: Validating job.
2016-03-18 14:31:31,759 [INFO] graphlab.deploy.map_job, 186: Validation complete. Job: 'Model-Parameter-Search-Mar-18-2016-14-31-3100001' ready for execution
2016-03-18 14:31:32,010 [INFO] graphlab.deploy.map_job, 192: Job: 'Model-Parameter-Search-Mar-18-2016-14-31-3100001' scheduled.
2016-03-18 14:31:32,389 [INFO] graphlab.deploy.job, 22: Validating job.
2016-03-18 14:31:32,399 [INFO] graphlab.deploy.map_job, 186: Validation complete. Job: 'Model-Parameter-Se

Wait until all the jobs have finished by wathcing CPU utilisation. Confirm the kernel is still running by executing the line below.

In [5]:
print 'still Ok'

still Ok


Now get the job status. There will be a 10s+ pause then the error message "The kernel appears to have died. It will restart automatically."

In [8]:
job.get_status()

{'Canceled': 0,
 'Completed': 11,
 'Failed': 0,
 'Pending': 36,
 'Running': 3,
 'Total': 50}

In [ ]:
job.progress_bar()